In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\korean_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'korean']

In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,korean
base_date,,
2018-01-31,50110250,7.209306e+06
2018-02-28,50110250,5.579125e+06
2018-03-31,50110250,7.615021e+06
2018-04-30,50110250,8.498349e+06
2018-05-31,50110250,8.916782e+06
...,...,...
2021-02-28,50130620,4.321265e+06
2021-03-31,50130620,4.744309e+06
2021-04-30,50130620,5.236706e+06


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110530  ##각행정동코드

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,korean
base_date,
2018-01-31,6.677128e+06
2018-02-28,5.923142e+06
2018-03-31,6.373473e+06
2018-04-30,6.265477e+06
2018-05-31,6.306896e+06
2018-06-30,6.052675e+06
2018-07-31,6.491371e+06
2018-08-31,6.632232e+06
2018-09-30,6.283526e+06


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'korean': base_date
 2018-01-31    6.677128e+06
 2018-02-28    5.923142e+06
 2018-03-31    6.373473e+06
 2018-04-30    6.265477e+06
 2018-05-31    6.306896e+06
 2018-06-30    6.052675e+06
 2018-07-31    6.491371e+06
 2018-08-31    6.632232e+06
 2018-09-30    6.283526e+06
 2018-10-31    6.463504e+06
 2018-11-30    6.339984e+06
 2018-12-31    6.501341e+06
 2019-01-31    6.644148e+06
 2019-02-28    6.153701e+06
 2019-03-31    7.397813e+06
 2019-04-30    8.328176e+06
 2019-05-31    7.629039e+06
 2019-06-30    7.107155e+06
 2019-07-31    7.668538e+06
 2019-08-31    7.474172e+06
 2019-09-30    6.977864e+06
 2019-10-31    7.209894e+06
 2019-11-30    6.495510e+06
 2019-12-31    7.281331e+06
 2020-01-31    7.419899e+06
 2020-02-29    6.972340e+06
 2020-03-31    6.631683e+06
 2020-04-30    6.330577e+06
 2020-05-31    6.811976e+06
 2020-06-30    6.835579e+06
 2020-07-31    7.183052e+06
 2020-08-31    7.010600e+06
 2020-09-30    6.525011e+06
 2020-10-31    6.829153e+06
 2020-11-30    7.372693e+06

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -3.367653
p-value : 0.012112
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1224.477, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1213.922, Time=0.10 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1215.192, Time=0.05 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1216.966, Time=0.05 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1221.319, Time=0.08 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1219.489, Time=0.09 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1211.684, Time=0.05 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1213.681, Time=0.05 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1215.018, Time=0.09 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1216.781, Time=0.09 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1220.648, Time=0.23 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1213.698, Time=0.03 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1271.806, Time=0.11 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1216.381, Time=0.20 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.26 sec
 ARIMA(3,0,0)(0

ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.korean.values, order=(1,0,0))   

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(1, 0)   Log Likelihood                -602.833
Method:                       css-mle   S.D. of innovations         412571.416
Date:                Sat, 11 Sep 2021   AIC                           1211.667
Time:                        01:53:28   BIC                           1216.880
Sample:                             0   HQIC                          1213.578
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       6.798e+06   1.34e+05     50.654      0.000    6.54e+06    7.06e+06
ar.L1.y        0.5383      0.126      4.262      0.000       0.291       0.786
                                    Roots           

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([6778607.65784879, 6787681.16517233]), array([412571.41563547, 468547.7310089 ]), array([[5969982.54215256, 7587232.77354501],
       [5869344.48735692, 7706017.84298773]]))
